# Command R+ tool/function calling using LangChain	

You can use Cohere Command R or Cohere Command R+ models deployed in Azure AI and Azure ML with `langchain` to create more sophisticated intelligent applications.

> Review the [documentation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command) for the Cohere family of models at for AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select `Cohere Command R` or `Cohere Command R+` and `Cohere-embed-v3-english` or `Cohere-embed-v3-multilingual`.
* Deploy with "Pay-as-you-go"

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

## Example

Tool use allows developers to connect Cohere's models to external tools like search engines, APIs, functions, databases, etc.

Multi-step tool use is an extension of this basic idea, and allows the model to call more than one tool in a sequence of steps, using the results from one tool call in a subsequent step. This process allows the language model to reason, perform dynamic actions, and quickly adapt on the basis of information coming from external sources.

The recommended way to achieve [multi-step tool use with Cohere](https://docs.cohere.com/docs/multi-step-tool-use) is by leveraging the [Langchain framework](https://python.langchain.com/docs/integrations/providers/cohere/#react-agent) in Python.


### Install Dependencies

In [ ]:
! pip install --quiet langchain langchain_cohere

In [ ]:
from langchain_cohere.chat_models import ChatCohere
from langchain_cohere import CohereEmbeddings
from langchain_core.documents.base import Document
from langchain.vectorstores import FAISS
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.pydantic_v1 import BaseModel, Field
import os

## Constants
We will set the values for the keys and models that we will use in our RAG pipeline

Let's create instances of our Chat and Embed models deployed in Azure AI or Azure ML. Use the `langchain_cohere` package and configure it as follows:

- `embed_endpoint` and `command_endpoint`: Use the endpoint URL from your deployment. Include `/v1` at the end of the endpoint URL.

In [ ]:
# Embed
azure_cohere_embed_endpoint = "https://<endpoint>.<region>.inference.ai.azure.com/v1"
azure_cohere_embed_key = "<key>"

# Command
azure_cohere_command_endpoint = "https://<endpoint>.<region>.inference.ai.azure.com/v1"
azure_cohere_command_key = "<key>"

In [ ]:
# Set up the command model to be used in a chat
chat_model = ChatCohere(
    base_url=azure_cohere_command_endpoint, cohere_api_key=azure_cohere_command_key
)

In [ ]:
# Set up the embedding model to be used in the vector index
embed_model = CohereEmbeddings(
    base_url=azure_cohere_embed_endpoint, cohere_api_key=azure_cohere_embed_key
)

### Define tools
Your agent will be equipped with the following tools. The model can pick between

* doing a search on the web
* and/or retrieving data from a vector store
* and/or directly answering [ this tool comes out of the box! ]

Plus the model can self-reflect.

### Vector search

You can easily equip your agent with a vector store!

In [ ]:
project_descriptions = [
    "Project Alpha: Implementing a new CRM system to improve customer relationships.",
    "Project Beta: Developing an AI-based forecasting tool for sales trends.",
    "Project Gamma: Overhauling the company website for better user engagement.",
    "Project Delta: Integrating blockchain technology for enhanced supply chain transparency.",
    "Project Epsilon: Launching a cloud migration strategy to enhance data accessibility and security.",
    "Project Zeta: Implementing machine learning algorithms to optimize inventory management systems.",
    "Project Eta: Developing a mobile application to improve field service operations and customer reporting.",
    "Project Theta: Upgrading cybersecurity measures to protect against emerging digital threats.",
    "Project Iota: Creating a virtual reality training program to enhance employee skill sets and engagement.",
    "Project Kappa: Automating financial processes using AI to increase efficiency and reduce errors.",
]

project_docs = []
for doc in project_descriptions:
    project_docs.append(Document(page_content=doc, metadata={"source": "local"}))

Ingest documents with Embed endpoint

In [ ]:
db = FAISS.from_documents(project_docs, embed_model)

We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [ ]:
db_retriever = db.as_retriever(search_kwargs={"k": 2})

In [ ]:
from langchain.tools.retriever import create_retriever_tool

vectorstore_search = create_retriever_tool(
    retriever=db_retriever,
    name="vectorstore_search",
    description="Retrieve relevant info from a vectorstore that contains information about Projects.",
)

### Web search
You can easily equip your agent with web search!

In [ ]:
os.environ[
    "TAVILY_API_KEY"
] = "API_KEY"  # you can create an API key for free on Tavily's website

internet_search = TavilySearchResults()
internet_search.name = "internet_search"
internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")


internet_search.args_schema = TavilySearchInput

## Create ReAct Agent

The model can smartly pick the right tool(s) for the user query, call them in any sequence, analyze the results and self-reflect.
Once the model considers it has enough information to answer the user question, it generates the final answer.

In [ ]:
# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource. You are equipped with an internet search tool and a special vectorstore of information about Projects.
Use internet search to define strategies.
"""

# Prompt template
prompt = ChatPromptTemplate.from_template("{input}")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=chat_model,
    tools=[internet_search, vectorstore_search],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent, tools=[internet_search, vectorstore_search], verbose=True
)

## Ask a standalone question to the ReAct agent
A question that requires using a predefined tool from Langchain

In [ ]:
response = agent_executor.invoke(
    {
        "input": "Summarize the AI based projects we are currently working on",
        "preamble": preamble,
    }
)

response["output"]

In [ ]:
response = agent_executor.invoke(
    {
        "input": "How can we automate financial processes on Project Kappa? Then define the strategy to execite that process.",
        "preamble": preamble,
    }
)

response["output"]